In [1]:
import warnings
warnings.simplefilter('ignore')

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# Red Recurrente (simple) con Numpy

In [2]:
import numpy as np


timesteps = 100     # numero de pasos de la secuencia de entrada
input_features = 32 # dimension espacio de caracteristicas de entrada
output_features = 64 # dimension de salida

inputs = np.random.random((timesteps, input_features)) # datos entrada

state_t = np.zeros((output_features,))  # estado inicial

In [3]:
# matrices de pesos y sesgo

W = np.random.random((output_features, input_features))

U = np.random.random((output_features, output_features))

b = np.random.random((output_features,))

In [4]:
successive_outputs = []

for input_t in inputs:
    # se combina output con nuevo input
    output_t = np.tanh(np.dot(W, input_t) + np.dot(U, state_t) + b)
    # se guarda en una lista
    successive_outputs.append(output_t)
    # se actualiza el estado inicial
    state_t = output_t

In [5]:
# (pasos, output)
final_output_sequence = np.concatenate(successive_outputs, axis=0)

In [6]:
final_output_sequence

array([0.99999978, 1.        , 1.        , ..., 1.        , 1.        ,
       1.        ])

# RNN con Keras

In [7]:
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import SimpleRNN


model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 32)                2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


Using TensorFlow backend.


In [8]:
# este modelo devuelve toda la secuencia

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))


model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, None, 32)          2080      
Total params: 322,080
Trainable params: 322,080
Non-trainable params: 0
_________________________________________________________________


In [9]:
# stacking de varias rnn

model = Sequential()
model.add(Embedding(10000, 32))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32, return_sequences=True))
model.add(SimpleRNN(32))



model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 32)          320000    
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_4 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_5 (SimpleRNN)     (None, None, 32)          2080      
_________________________________________________________________
simple_rnn_6 (SimpleRNN)     (None, 32)                2080      
Total params: 328,320
Trainable params: 328,320
Non-trainable params: 0
_________________________________________________________________


## Preparando datos

In [13]:
from keras.datasets import imdb
from keras.preprocessing import sequence

import numpy as np

In [14]:
max_features = 10000
maxlen = 500
batch_size = 32

In [16]:
print('Loading data...')

np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

(input_train, y_train), (input_test, y_test) = imdb.load_data(
    num_words=max_features)

np.load = np_load_old


print(len(input_train), 'train sequences')
print(len(input_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [17]:
print('Pad sequences (samples x time)')

input_train = sequence.pad_sequences(input_train, maxlen=maxlen)
input_test = sequence.pad_sequences(input_test, maxlen=maxlen)


print('input_train shape:', input_train.shape)
print('input_test shape:', input_test.shape)

Pad sequences (samples x time)
input_train shape: (25000, 500)
input_test shape: (25000, 500)


**entrenando un modelo**

In [ ]:
from keras.layers import Dense


model = Sequential() 
model.add(Embedding(max_features, 32)) 
model.add(SimpleRNN(32)) 
model.add(Dense(1, activation='sigmoid'))


model.compile(optimizer='rmsprop', 
              loss='binary_crossentropy', 
              metrics=['acc']) 




history = model.fit(input_train, y_train,
                    epochs=10, batch_size=128,
                    validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
 2816/20000 [===>..........................] - ETA: 11s - loss: 0.6928 - acc: 0.5210

**plots**

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline



acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']


epochs = range(1, len(acc) + 1)


plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()


plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()


plt.show();